How to start a webdriver client in node?

In [ ]:
var importer = require('../Core');
var sync = require('wdio-sync')
var readSessions = importer.import('load webdriver sessions');

var TIMEOUT = 10000;
var sessionSync = false;

var client;
function createWebdriverClient(host, port) {
    if(sessionSync) {
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => createWebdriverClient(host, port));
    }
    sessionSync = true;
    var sessions = readSessions();
    
    var webdriverServer = {
//        sync: true,
        debug: true,
        host: host || 'localhost',
        port: port || 4444,
        logLevel: 'command',
        baseUrl: 'https://webdriver.io',
        pageLoadStrategy: 'eager',
        connectionRetryTimeout: TIMEOUT,
        desiredCapabilities: {
            browserName: 'chrome',
            chromeOptions: {
                prefs: {
                    'download.default_directory': '/data/downloads',
                    'profile.default_content_setting_values.notifications': 2,
                    'exited_cleanly': true,
                    'exit_type': 'None'
                },
                args: [
                    // TODO: https://superuser.com/questions/461035/disable-google-chrome-session-restore-functionality
                    'user-data-dir=/tmp/profile-' + sessions.length % 36,
                    // 'start-fullscreen',
                    'no-sandbox',
                    'disable-session-crashed-bubble',
                    'disable-infobars',
                    'new-window',
                    'disable-geolocation',
                    'disable-notifications',
                    'show-saved-copy',
                    'silent-debugger-extension-api'
                    //'kiosk'
                ]
            }
        },
    };
    
    console.log('Initializing webdriver on ' + webdriverServer.host);
    client = require('webdriverio').remote(webdriverServer);
    const {
        updateOrAddSession,
        getSessions,
    } = importer.import('manage webdriver sessions', {client});
    client.on('error', e => { 
        console.log(e.message);
    });
    client.on('end', () => console.log('Daemon: Closing browser'));
    client.on('result', (result) => updateOrAddSession(client.requestHandler.sessionID));
    
    return client
        .getSessions(true)
        // save current session
        .then(validSessions => {
            // the next null or end will be the next available profile id
            var index = sessions.map(s => s[1]).indexOf(validSessions[0] || null);
            if(index === -1) {
                index = sessions.length;
            }
            client.options.desiredCapabilities.chromeOptions.args[0] = 'user-data-dir=/tmp/profile-' + index;
            // TODO: fix this, doesn't work on second init, keeps opening new windows if chrome profile path is alreading open for read/write
            return client
                .then(() => (client.requestHandler.sessionID = validSessions[0] || null)
                      || client.init())
                .session();
        })
        .then(r => {
            sessions = updateOrAddSession(client.requestHandler.sessionID);
            console.log(client.requestHandler.sessionID);
        })
        .catch(e => console.log(e))
        // Down here at the bottom for safetey
        .then(() => {
            sessionSync = false;
            return client
        })
};
module.exports = createWebdriverClient;
createWebdriverClient;


Load webdriver sessions?


In [ ]:
var fs = require('fs');
var path = require('path');

var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

var sessions = {};
var sessionModified = 0;

function readSessions() {
    try {
        if(fs.existsSync(SESSIONS_PATH)
           && fs.statSync(SESSIONS_PATH).mtime.getTime() > sessionModified) {
            sessionModified = fs.statSync(SESSIONS_PATH).mtime.getTime();
            sessions = JSON.parse(fs.readFileSync(SESSIONS_PATH)
                .toString());
        }
    } catch (e) {
        sessions = [];
    }
    return sessions;
};
module.exports = readSessions;


Manage webdriver sessions?



In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var readSessions = importer.import('load webdriver sessions');

var TIMEOUT = 10000;
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

function updateOrAddSession(currentSession) {
    const sessions = readSessions();
    const updateSession = sessions.filter(s => s[1] === currentSession)[0];
    if(typeof updateSession !== 'undefined') {
        if((new Date()).getTime() - updateSession[0] > TIMEOUT / 2) {
            updateSession[0] = (new Date()).getTime();
        } else {
            return;
        }
    } else {
        const oldSession = sessions.filter(s => s[1] === null)[0] || (sessions[sessions.length] = []);
        // http://www.english.upenn.edu/~jenglish/English104/tzara.html
        oldSession[1] = currentSession;
        oldSession[0] = (new Date()).getTime();
    }
    fs.writeFileSync(
        SESSIONS_PATH,
        JSON.stringify(sessions, null, 4));
    return sessions;
}

function getSessions(inactive = false) {
    const sessions = readSessions();
    const original = client.requestHandler.sessionID;
    var active = [].concat(sessions)
        .filter(session => typeof session[1] !== 'undefined' && session[1] !== null);
    if(inactive) {
        active = active.filter(session => (new Date()).getTime() - session[0] > TIMEOUT);
    }
    return importer.runAllPromises(active.map(session => (resolve) => {
        client.requestHandler.sessionID = session[1];
        client
            .windowHandle()
            .then(r => client.window(r.value))
            .session()
            .then(s => resolve(s.sessionId))
            .catch(e => {
                // if the session is really old and has an error delete it from the list
                const index = sessions.map(s => s[1]).indexOf(session[1]);
                sessions[index][1] = null;
                resolve(null);
            });
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            if(original) {
                updateOrAddSession(original);
            }
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

if(typeof client.getSessions === 'undefined') {
    client.addCommand('getSessions', getSessions);
}

module.exports = {
    updateOrAddSession,
    getSessions
}



In [ ]:
$$.async();
var client = createWebdriverClient('localhost', 4444)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost


In [ ]:
$$.async();
client.windowHandles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


How to end the webdriver service?

In [ ]:
client.endAll();



TODO: add decorated logging with screenshots of buttons results can be used: https://github.com/megamindbrian/bots/blob/master/bots/server.js

TODO: transfer state and cache to client

